In [1]:
import pandas as pd
import numpy as np

import xgboost
import lightgbm

import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as st
import time
import timeit

In [6]:
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, make_scorer
from sklearn.metrics import mean_squared_error

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [3]:
import sklearn.datasets as skds

mnist = skds.load_digits()
faces = skds.fetch_olivetti_faces()

In [4]:
x_mnist, y_mnist = mnist.data, mnist.target
x_faces, y_faces = faces.data, faces.target

In [7]:
classifiers = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'XGBoost': XGBClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Naive Bayes': GaussianNB(),
    'Neural Network': MLPClassifier(max_iter=500)
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [8]:
def stratified_cross_val_scores(clf, X, y, cv):
    accuracy_scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    precision_scores = cross_val_score(clf, X, y, cv=cv, scoring='precision_macro')
    recall_scores = cross_val_score(clf, X, y, cv=cv, scoring='recall_macro')
    return accuracy_scores, precision_scores, recall_scores


In [9]:
def experiment(dataset_name, X, y, classifiers, cv):
    results = []
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    print(f"Evaluating models on {dataset_name} dataset:")
    for clf_name, clf in classifiers.items():
        print(f"  Evaluating {clf_name}...")
        start_time = time.time()
        if clf_name in ['Random Forest', 'XGBoost', 'AdaBoost']:
            clf_ovo = OneVsOneClassifier(clf)
            clf_ovr = OneVsRestClassifier(clf)
            accuracy_scores_ovo, precision_scores_ovo, recall_scores_ovo = stratified_cross_val_scores(clf_ovo, X_train, y_train, cv)
            accuracy_scores_ovr, precision_scores_ovr, recall_scores_ovr = stratified_cross_val_scores(clf_ovr, X_train, y_train, cv)
            runtime = time.time() - start_time
            
            results.append({
                'Dataset': dataset_name,
                'Classifier': clf_name + ' (One-vs-One)',
                'Accuracy Mean': np.mean(accuracy_scores_ovo),
                'Accuracy CI': np.std(accuracy_scores_ovo) * 2,
                'Precision Mean': np.mean(precision_scores_ovo),
                'Precision CI': np.std(precision_scores_ovo) * 2,
                'Recall Mean': np.mean(recall_scores_ovo),
                'Recall CI': np.std(recall_scores_ovo) * 2,
                'Runtime': runtime
            })
            
            results.append({
                'Dataset': dataset_name,
                'Classifier': clf_name + ' (One-vs-Rest)',
                'Accuracy Mean': np.mean(accuracy_scores_ovr),
                'Accuracy CI': np.std(accuracy_scores_ovr) * 2,
                'Precision Mean': np.mean(precision_scores_ovr),
                'Precision CI': np.std(precision_scores_ovr) * 2,
                'Recall Mean': np.mean(recall_scores_ovr),
                'Recall CI': np.std(recall_scores_ovr) * 2,
                'Runtime': runtime
            })
        else:
            accuracy_scores, precision_scores, recall_scores = stratified_cross_val_scores(clf, X_train, y_train, cv)
            runtime = time.time() - start_time
            
            results.append({
                'Dataset': dataset_name,
                'Classifier': clf_name,
                'Accuracy Mean': np.mean(accuracy_scores),
                'Accuracy CI': np.std(accuracy_scores) * 2,
                'Precision Mean': np.mean(precision_scores),
                'Precision CI': np.std(precision_scores) * 2,
                'Recall Mean': np.mean(recall_scores),
                'Recall CI': np.std(recall_scores) * 2,
                'Runtime': runtime
            })
    return results

results_mnist = experiment('MNIST', x_mnist, y_mnist, classifiers, cv)
results_faces = experiment('Olivetti Faces', x_faces, y_faces, classifiers, cv)


results_df_mnist = pd.DataFrame(results_mnist)
results_df_faces = pd.DataFrame(results_faces)


Evaluating models on MNIST dataset:
  Evaluating Logistic Regression...


/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

  Evaluating Random Forest...
  Evaluating SVM...
  Evaluating K-Nearest Neighbors...
  Evaluating Decision Tree...
  Evaluating XGBoost...
  Evaluating AdaBoost...
  Evaluating Naive Bayes...
  Evaluating Neural Network...
Evaluating models on Olivetti Faces dataset:
  Evaluating Logistic Regression...


/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

  Evaluating Random Forest...


...

/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_cl

  Evaluating Naive Bayes...


/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_cl

  Evaluating Neural Network...


/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/bora/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/bora/anaconda3/lib/python3.11/sit

In [10]:
print("Results for MNIST Dataset:")
print(results_df_mnist)

print("\nResults for Olivetti Faces Dataset:")
print(results_df_faces)

Results for MNIST Dataset:
   Dataset                   Classifier  Accuracy Mean  Accuracy CI  \
0    MNIST          Logistic Regression       0.961735     0.018061   
1    MNIST   Random Forest (One-vs-One)       0.965907     0.015459   
2    MNIST  Random Forest (One-vs-Rest)       0.970775     0.023529   
3    MNIST                          SVM       0.988165     0.007126   
4    MNIST          K-Nearest Neighbors       0.987466     0.015021   
5    MNIST                Decision Tree       0.843460     0.051065   
6    MNIST         XGBoost (One-vs-One)       0.949211     0.028321   
7    MNIST        XGBoost (One-vs-Rest)       0.966594     0.022304   
8    MNIST        AdaBoost (One-vs-One)       0.956153     0.012990   
9    MNIST       AdaBoost (One-vs-Rest)       0.950595     0.025053   
10   MNIST                  Naive Bayes       0.822554     0.018935   
11   MNIST               Neural Network       0.974949     0.017278   

    Precision Mean  Precision CI  Recall Mean  Re

### Results for Mnist Dataset

| Dataset | Classifier                     | Accuracy Mean | Accuracy CI | Precision Mean | Precision CI | Recall Mean | Recall CI | Runtime   |
|---------|--------------------------------|---------------|-------------|----------------|--------------|-------------|-----------|-----------|
| MNIST   | Logistic Regression            | 0.961735      | 0.018061    | 0.963323       | 0.016367     | 0.961730    | 0.018107  | 0.824478  |
| MNIST   | Random Forest (One-vs-One)     | 0.965907      | 0.015459    | 0.971362       | 0.017309     | 0.963755    | 0.029852  | 42.102012 |
| MNIST   | Random Forest (One-vs-Rest)    | 0.970775      | 0.023529    | 0.976055       | 0.013147     | 0.972752    | 0.019865  | 42.102012 |
| MNIST   | SVM                            | 0.988165      | 0.007126    | 0.988531       | 0.006777     | 0.988100    | 0.006897  | 0.491915  |
| MNIST   | K-Nearest Neighbors            | 0.987466      | 0.015021    | 0.987954       | 0.014304     | 0.987412    | 0.014939  | 0.321542  |
| MNIST   | Decision Tree                  | 0.843460      | 0.051065    | 0.848712       | 0.059080     | 0.845248    | 0.029988  | 0.144568  |
| MNIST   | XGBoost (One-vs-One)           | 0.949211      | 0.028321    | 0.951438       | 0.027108     | 0.949463    | 0.028208  | 7.702895  |
| MNIST   | XGBoost (One-vs-Rest)          | 0.966594      | 0.022304    | 0.967081       | 0.021606     | 0.966494    | 0.022083  | 7.702895  |
| MNIST   | AdaBoost (One-vs-One)          | 0.956153      | 0.012990    | 0.957531       | 0.011558     | 0.956167    | 0.012537  | 33.083808 |
| MNIST   | AdaBoost (One-vs-Rest)         | 0.950595      | 0.025053    | 0.951988       | 0.024341     | 0.950650    | 0.025224  | 33.083808 |
| MNIST   | Naive Bayes                    | 0.822554      | 0.018935    | 0.868671       | 0.015119     | 0.823013    | 0.019429  | 0.022127  |
| MNIST   | Neural Network                 | 0.974949      | 0.017278    | 0.973630       | 0.019731     | 0.974151    | 0.003393  | 6.036316  |


### Results for Olivetti Faces Dataset

| Dataset        | Classifier                   | Accuracy Mean | Accuracy CI | Precision Mean | Precision CI | Recall Mean | Recall CI | Runtime      |
|----------------|------------------------------|---------------|-------------|----------------|--------------|-------------|-----------|--------------|
| Olivetti Faces | Logistic Regression          | 0.946875      | 0.046771    | 0.959167       | 0.047258     | 0.9525      | 0.045826  | 28.887076    |
| Olivetti Faces | Random Forest (One-vs-One)   | 0.909375      | 0.045928    | 0.904083       | 0.057591     | 0.9000      | 0.086603  | 526.392026   |
| Olivetti Faces | Random Forest (One-vs-Rest)  | 0.875000      | 0.065551    | 0.875833       | 0.109848     | 0.8625      | 0.090830  | 526.392026   |
| Olivetti Faces | SVM                          | 0.903125      | 0.066732    | 0.899583       | 0.082344     | 0.9075      | 0.060415  | 4.656062     |
| Olivetti Faces | K-Nearest Neighbors          | 0.768750      | 0.060596    | 0.764333       | 0.066789     | 0.7875      | 0.068920  | 0.256171     |
| Olivetti Faces | Decision Tree                | 0.515625      | 0.083853    | 0.497583       | 0.109225     | 0.5700      | 0.037417  | 19.432330    |
| Olivetti Faces | XGBoost (One-vs-One)         | 0.484375      | 0.106434    | 0.430798       | 0.129923     | 0.4900      | 0.091378  | 481.331188   |
| Olivetti Faces | XGBoost (One-vs-Rest)        | 0.771875      | 0.096014    | 0.736250       | 0.123029     | 0.7800      | 0.117898  | 481.331188   |
| Olivetti Faces | AdaBoost (One-vs-One)        | 0.775000      | 0.093958    | 0.769389       | 0.136000     | 0.7550      | 0.152151  | 1176.410334  |
| Olivetti Faces | AdaBoost (One-vs-Rest)       | 0.746875      | 0.023385    | 0.748208       | 0.096875     | 0.7675      | 0.056125  | 1176.410334  |
| Olivetti Faces | Naive Bayes                  | 0.809375      | 0.095607    | 0.814000       | 0.097827     | 0.8250      | 0.103682  | 0.344856     |
| Olivetti Faces | Neural Network               | 0.543750      | 0.730315    | 0.548298       | 0.342853     | 0.7100      | 0.665094  | 82.198577    |


### Summary

**Logistic Regression** performed consistently and gave very well results for both datasets with a slight drop in performance and increased runtime for the Olivetti Faces dataset.

**Decision Tree** gave good results for Mnist dataset but showed a really high drop in performance for Olivetti Faces dataset with an increase in runtime.

**SVM** showed high performance across both datasets but with a noticeable drop in metrics and increased runtime for Olivetti Faces.

**KNN** gave really high score similar to **SVM** for MNIST dataset but not as precise as **SVM**. For Olivetti Faces, there was a big drop in accuracy.

**XGBoost (OvO)** has a dramatic decrease in performance and increase in runtime for the Olivetti Faces dataset.

**Random Forest (OvR)** perform well for both datasets but with a drop in accuracy, precision, recall, and a significant increase in runtime for the Olivetti Faces dataset.

**Neural Network** gave much better results than Olivetti Dataset on the Mnist Dataset.

#### In general, the algorithms took longer to give results and not as accurate as Mnist for the Olivetti Faces Dataset. This may be due to Olivetti's more complex structure.

**Best Model for MNIST:**

SVM has the highest accuracy, precision, and recall, with a very low runtime and low confidence intervals, shows that it is the most reliable model for the MNIST dataset.



**Best Model for Olivetti Faces:**

Logistic Regression has the highest accuracy, precision, and recall, with fair runtime and confidence intervals, making it the most reliable model for the Olivetti Faces dataset. 

https://chatgpt.com/share/e97a00bd-794c-4cf5-936f-5516bec852cc